# Exponential smoothing 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
plt.rcParams["figure.figsize"] = (9,6)

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['value'])
ax.set_xlabel('Date')
ax.set_ylabel('Monthly government expenditure (millions)')

plt.xticks(np.arange(6, 204, 12), np.arange(1992, 2009, 1))

plt.title('Monthly corticosteroid drug subsidy in Australia from 1991 to 2008')

fig.autofmt_xdate()
plt.tight_layout()

## Simple exponential smoothing

In [ ]:
# Train/test split (last 36 time steps for the test set)



In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

def rolling_forecast(df, train_len, horizon, window, method):
    
    total_len = train_len + horizon
    end_idx = train_len

    if method == 'last_season':
        pred_last_season = []
        
        for i in range(train_len, total_len, window):
            last_season = df[:i].iloc[-12:].values
            pred_last_season.extend(last_season)
            
        return pred_last_season
    
    elif method == 'simple_exp':
        # Get the predictions from simple exponential smoothing

        return pred_simple_exp

In [ ]:
TRAIN_LEN = len(train)
HORIZON = len(test)
WINDOW = 12

pred_last_season = rolling_forecast(df['value'], TRAIN_LEN, HORIZON, WINDOW, 'last_season')
pred_simple_exp = rolling_forecast(df['value'], TRAIN_LEN, HORIZON, WINDOW, 'simple_exp')

test.loc[:, 'pred_last_season'] = pred_last_season
test.loc[:, 'pred_simple_exp'] = pred_simple_exp

test.head()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['value'])
ax.plot(test['value'], 'b-', label='Actual')
ax.plot(test['pred_last_season'], 'r:', label='naive seasonal')
ax.plot(test['pred_simple_exp'], 'g-.', label='SES')
ax.set_xlabel('Date')
ax.set_ylabel('Monthly government expenditure (millions)')
ax.axvspan(169, 204, color='#808080', alpha=0.2)
ax.legend(loc='best')

plt.xticks(np.arange(6, 204, 12), np.arange(1992, 2009, 1))

plt.title('Monthly corticosteroid drug subsidy in Australia from 1991 to 2008')

fig.autofmt_xdate()
plt.tight_layout()

### Evaluation 

In [ ]:
from sklearn.metrics import mean_absolute_error

baseline_mae = mean_absolute_error(test['value'], test['pred_last_season'])
ses_mae = mean_absolute_error(test['value'], test['pred_simple_exp'])

print(baseline_mae, ses_mae)

In [ ]:
fig, ax = plt.subplots()

x = ['naive seasonal', 'SES']
y = [baseline_mae, ses_mae]

ax.bar(x, y, width=0.4)
ax.set_xlabel('Models')
ax.set_ylabel('MAE')
ax.set_ylim(0, 0.5)

for index, value in enumerate(y):
    plt.text(x=index, y=value + 0.025, s=str(round(value,2)), ha='center')

plt.tight_layout()

## Double exponential smoothing 

In [ ]:
from statsmodels.tsa.holtwinters import Holt

def rolling_forecast(df, train_len, horizon, window, method):
    
    total_len = train_len + horizon
    end_idx = train_len

    if method == 'last_season':
        pred_last_season = []
        
        for i in range(train_len, total_len, window):
            last_season = df[:i].iloc[-12:].values
            pred_last_season.extend(last_season)
            
        return pred_last_season
    
    elif method == 'simple_exp':
        pred_simple_exp = []
        
        for i in range(train_len, total_len, window):
            ses = SimpleExpSmoothing(df[:i], initialization_method='estimated').fit()
            predictions = ses.forecast(window)
            pred_simple_exp.extend(predictions)
            
        return pred_simple_exp
    
    elif method == 'double_exp':
        # Get predictions from DES
    
    elif method == 'double_exp_damp':
         # Get predictions from DES with damping
        
        return pred_double_exp_damp

In [ ]:
pred_double_exp = rolling_forecast(df['value'], TRAIN_LEN, HORIZON, WINDOW, 'double_exp')
pred_double_exp_damp = rolling_forecast(df['value'], TRAIN_LEN, HORIZON, WINDOW, 'double_exp_damp')

test.loc[:, 'pred_double_exp'] = pred_double_exp
test.loc[:, 'pred_double_exp_damp'] = pred_double_exp_damp

test.head()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['value'])
ax.plot(test['value'], 'b-', label='Actual')
ax.plot(test['pred_last_season'], 'r:', label='naive seasonal')
ax.plot(test['pred_simple_exp'], 'g-.', label='SES')
ax.plot(test['pred_double_exp'], 'k--', label='DES')
ax.plot(test['pred_double_exp_damp'], color='orange', label='DES - damped')
ax.set_xlabel('Date')
ax.set_ylabel('Monthly government expenditure (millions)')
ax.axvspan(169, 204, color='#808080', alpha=0.2)
ax.legend(loc='best')


plt.xticks(np.arange(6, 204, 12), np.arange(1992, 2009, 1))

plt.title('Monthly corticosteroid drug subsidy in Australia from 1991 to 2008')

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
des_mae = mean_absolute_error(test['value'], test['pred_double_exp'])
des_damp_mae = mean_absolute_error(test['value'], test['pred_double_exp_damp'])

In [ ]:
fig, ax = plt.subplots()

x = ['naive seasonal', 'SES', 'DES', 'DES - damped']
y = [baseline_mae, ses_mae, des_mae, des_damp_mae]

ax.bar(x, y, width=0.4)
ax.set_xlabel('Models')
ax.set_ylabel('MAE')
ax.set_ylim(0, 0.5)

for index, value in enumerate(y):
    plt.text(x=index, y=value + 0.025, s=str(round(value,2)), ha='center')

plt.tight_layout()

## Triple exponential smoothing

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

def rolling_forecast(df: pd.DataFrame, train_len: int, horizon: int, window: int, method: str) -> list:
    
    total_len = train_len + horizon
    end_idx = train_len

    if method == 'last_season':
        pred_last_season = []
        
        for i in range(train_len, total_len, window):
            last_season = df[:i].iloc[-12:].values
            pred_last_season.extend(last_season)
            
        return pred_last_season
    
    elif method == 'simple_exp':
        pred_simple_exp = []
        
        for i in range(train_len, total_len, window):
            ses = SimpleExpSmoothing(df[:i], initialization_method='estimated').fit()
            predictions = ses.forecast(window)
            pred_simple_exp.extend(predictions)
            
        return pred_simple_exp
    
    elif method == 'double_exp':
        pred_double_exp = []
        
        for i in range(train_len, total_len, window):
            des = Holt(df[:i], initialization_method='estimated').fit()
            predictions = des.forecast(window)
            pred_double_exp.extend(predictions)
            
        return pred_double_exp
    
    elif method == 'double_exp_damp':
        pred_double_exp_damp = []
        
        for i in range(train_len, total_len, window):
            des_damp = Holt(df[:i],damped_trend=True, initialization_method='estimated').fit()
            predictions = des_damp.forecast(window)
            pred_double_exp_damp.extend(predictions)
            
        return pred_double_exp_damp
    
    elif method == 'triple_exp':
        # Get prediction from TES

        return pred_triple_exp

In [ ]:
pred_triple_exp = rolling_forecast(df['value'], TRAIN_LEN, HORIZON, WINDOW, 'triple_exp')

test.loc[:, 'pred_triple_exp'] = pred_triple_exp

test.head()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['value'])
ax.plot(test['value'], 'b-', label='Actual')
ax.plot(test['pred_last_season'], 'r:', label='naive seasonal')
ax.plot(test['pred_simple_exp'], 'g-.', label='SES')
ax.plot(test['pred_double_exp'], 'k--', label='DES')
ax.plot(test['pred_double_exp_damp'], color='orange', label='DES - damped')
ax.plot(test['pred_triple_exp'], color='yellow', label='TES')
ax.set_xlabel('Date')
ax.set_ylabel('Monthly government expenditure (millions)')
ax.axvspan(169, 204, color='#808080', alpha=0.2)
ax.legend(loc='best')


plt.xticks(np.arange(6, 204, 12), np.arange(1992, 2009, 1))

plt.title('Monthly corticosteroid drug subsidy in Australia from 1991 to 2008')

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
tes_mae = mean_absolute_error(test['value'], test['pred_triple_exp'])

In [ ]:
fig, ax = plt.subplots()

x = ['naive seasonal', 'SES', 'DES', 'DES - damped', 'TES']
y = [baseline_mae, ses_mae, des_mae, des_damp_mae, tes_mae]

ax.bar(x, y, width=0.4)
ax.set_xlabel('Models')
ax.set_ylabel('MAE')
ax.set_ylim(0, 0.5)

for index, value in enumerate(y):
    plt.text(x=index, y=value + 0.025, s=str(round(value,2)), ha='center')

plt.tight_layout()